In [1]:
__author__ = 'Alice Jacques <alice.jacques@noao.edu>, NOIRLab Astro Data Lab Team <datalab@noao.edu>'
__version__ = '20200904'
__keywords__ = ['vospace','mydb','store files','query']

# How to use the Data Lab *Command Line Client* Service

### Table of Contents

* [Summary](#summary)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#imports)
* [Handling VOSpace directories/files via the datalab command line client](#datalabcommand)
    - [Uploading a file](#cmdupload)
    - [Downloading a file](#cmddownload)
    - [Copying a file/directory](#cmdcopy)
    - ~[Linking a file/directory](#cmdlink)~
    - [Creating a directory](#cmdcreate)
    - [Moving a file/directory](#cmdmove)
    - [Deleting a file](#cmddeletefile)
    - [Deleting a directory](#cmddeletedirectory)
    - ~[Tagging a file/directory](#cmdtag)~
* [Handling MyDB tables via the datalab command line client](#datalabcmdmydb)
    - [Listing MyDB tables and a table's schema](#listcmdmydb)
    - [Creating a MyDB table](#createcmdmydb)
    - [Inserting data into a MyDB table](#insertcmdmydb)
    - [Importing data into a MyDB table](#importcmdmydb)
    - [Truncating a MyDB table](#truncmdmydb)
    - [Copying a MyDB table](#copycmdmydb)
    - [Renaming a MyDB table](#renamecmdmydb)
    - [Dropping a MyDB table](#dropcmdmydb)
    
*Note: those that are crossed out above indicate this feature is currently not working.*    

<a class="anchor" id="summary"></a>
# Summary

This notebook documents how to use the Data Lab virtual storage system VOSpace and a user's personal MyDB database via the command line using the *datalab* command. The *datalab* command provides an alternate command line way to work with the auth client, query client, and store client. The API documentation can be found [here](https://datalab.noao.edu/docs/manual/UsingTheNOAODataLab/CommandLineTools/TheDatalabCommand/TheDatalabCommand.html).

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: http://datalab.noao.edu/disclaimers.php

<a class="anchor" id="imports"></a>
# Imports & setup

In [2]:
from dl import queryClient as qc
from dl.helpers.utils import convert

The Data Lab Command Line Client (DCLC) is a Python-based package that provides an alternate way to interact with the various Data Lab services. It can be installed with 

    pip install --ignore-installed --no-cache-dir noaodatalab
    
It is invoked via the datalab command. 

We need to be logged into the Data Lab to use the query client and store client. Enter your Data Lab username after '*user=*' and enter your password for Data Lab after '*password=*' below:

In [3]:
!datalab login user=ajacques password=

User 'ajacques' is already logged in to the Data Lab


<a class="anchor" id="datalabcommand"></a>
# 1. Handling directories/files via the datalab command line client

The *datalab* command provides a way to use a user's VOSpace. VOSpace is a convenient virtual storage space for users to save their work. It can store any data or file type.

Before we start this section, let's first query some example data from a Data Lab database and save it locally as a CSV file named `smags.csv`:

In [51]:
query = 'SELECT gmag, imag, rmag, zmag FROM smash_dr1.object LIMIT 10'
qc.query(adql=query,fmt='csv',out='./smags.csv')

'OK'

<a class="anchor" id="cmdupload"></a>
### 1.1 Uploading a file

Let's say we want to upload a file from our local disk to the virtual storage:

In [52]:
!datalab put fr="./smags.csv" to="vos://smags.csv"

<a class="anchor" id="cmddownload"></a>
### 1.2 Downloading a file

Let's say we want to download a file from our virtual storage space, in this case the csv file that we uploaded to it in the last cell:

In [53]:
!datalab get fr="vos://smags.csv" to="./mysmags.csv"

<a class="anchor" id="cmdcopy"></a>
### 1.3 Copying a file/directory

We want to put a copy of the file in a remote work directory:

In [54]:
!datalab cp fr="vos://smags.csv" to="vos://tmp/smags.csv"

<a class="anchor" id="cmdlink"></a>
### ~1.4 Linking to a file/directory~
**WARNING**: Linking is currently **not** working in the Data Lab storage manager. This notebook will be updated when the problem has been resolved.

Sometimes we want to create a link to a file or directory:

In [ ]:
#!datalab ln fr="vos://temp/mags.csv" to="vos://mags.csv"

<a class="anchor" id="cmdlist"></a>
### 1.5 Listing a file/directory

We can see all the files that are in a specific directory or get a full listing for a specific file:

In [59]:
!datalab ls name="vos://tmp"

mags.csv


<a class="anchor" id="cmdcreate"></a>
### 1.6 Creating a directory

We can create a directory:

In [56]:
!datalab mkdir name="vos://results"

<a class="anchor" id="cmdmove"></a>
### 1.7 Moving a file/directory

We can move a file or directory:

In [57]:
!datalab mv fr="vos://tmp/smags.csv" to="vos://results"

<a class="anchor" id="cmddeletefile"></a>
### 1.8 Deleting a file

We can delete a file:

In [33]:
!datalab rm name="vos://results/smags.csv"

<a class="anchor" id="cmddeletedirectory"></a>
### 1.9 Deleting a directory

We can also delete a directory:

In [60]:
!datalab rmdir name="vos://results"

<a class="anchor" id="cmdtag"></a>
### ~1.10 Tagging a file/directory~
**Warning**: Tagging is currently **not** working in the Data Lab storage manager. This notebook will be updated when the problem has been resolved.

We can tag any file or directory with arbitrary metadata:

In [ ]:
#!datalab tag name="vos://results" tag="The results from my analysis"

<a class="anchor" id="datalabcmdmydb"></a>
# 2. Handling MyDB tables via the datalab command line client
The *datalab* command provides a way to use a user's MyDB database. MyDB is a useful virtual storage space for users to save their work as a table. It can only store data tables. **_NOTE: The data must be in the form of either a CSV file or Pandas Dataframe object in order to load it into MyDB._**

<a class="anchor" id="listcmdmydb"></a>
### 2.1 Listing MyDB tables and a table's schema

We can list all of the MyDB tables currently in a user's database with the *mydb_list* function:

In [3]:
!datalab mydb_list

c4d,created:2020-08-27 14:55:07 MST
cd,created:2020-08-27 15:08:40 MST
gaia_result_table,created:2020-09-04 15:59:48 MST
quickresults,created:2020-08-25 14:44:34 MST
quickresults2,created:2020-08-25 14:46:01 MST
usno_objects,created:2020-09-02 12:17:52 MST



We can also list the schema and schema's datatype in a specified MyDB table:

In [ ]:
!datalab mydb_list table="usno_objects"

<a class="anchor" id="createcmdmydb"></a>
### 2.2 Creating a MyDB table 
We can create a new empty MyDB table with a user-provided schema file using the *mydb_create* function with the following parameters:

*table* - name of the new MyDB table to create  
*schema* - location and name of the schema definition to be in the table

The schema definition is stored in a text file, in this case in the user notebook directory. The schema definition file is a CSV-formatted file that contains column name and (Postgres) data type, one row per column. The general format is:

`Columnname1,datatype1\nColumnname2,datatype2\nColumnname3,datatype3`

Let's first create a simple (id,ra,dec) schema of a text string and two double values and save it locally as a text file named `schema.txt`:

In [ ]:
schema_str = 'id,text\nra,double precision\ndec,double precision\n'
with open ('schema.txt','w') as fd:
    fd.write (schema_str)

Now let's use the *mydb_create* function to make a new table in MyDB with the schema definition we created above:

In [5]:
!datalab mydb_create table="createdtable" schema="./schema.txt"

OK


Let's make sure the table was created in MyDB by calling the *mydb_list* function:

In [6]:
!datalab mydb_list

c4d,created:2020-08-27 14:55:07 MST
cd,created:2020-08-27 15:08:40 MST
createdtable,created:2020-09-04 17:00:56 MST
gaia_result_table,created:2020-09-04 15:59:48 MST
quickresults,created:2020-08-25 14:44:34 MST
quickresults2,created:2020-08-25 14:46:01 MST
usno_objects,created:2020-09-02 12:17:52 MST



Let's also make sure the schema was loaded into the table by calling the *mydb_list* function on the table:

In [7]:
!datalab mydb_list table="createdtable"

id,text,
ra,double precision,
dec,double precision,



<a class="anchor" id="insertcmdmydb"></a>
### 2.3 Inserting data into a MyDB table ???? 
**This is the same as import so..... is it relevant to have? Also, when the last time I ran the 'mydb_insert' cell below it froze and would not let me stop the kernal...**  

    class MyDB_Insert(Task):
    '''
        Insert data into a user MyDB table.
    '''
    def __init__(self, datalab):
        Task.__init__(self, datalab, 'mydb_insert',
                      'Insert data into a user MyDB table')
        self.addOption("table",
            Option("table", "", "Table name to create", required=True))
        self.addOption("data",
            Option("data", "", "Data file to load", required=True))
        self.addStdOptions()

    def run(self):
        token = getUserToken(self)
        try:
            res = queryClient.mydb_import (token, self.table.value,
                                           self.data.value)
        except Exception as e:
            print ("Error importing table '%s': %s" % (self.table.value,str(e)))
        else:
            print (res)

We can choose to insert data saved on a local computer or insert data from VOSpace into a pre-existing MyDB table. The data must be in the form of either a CSV file or Pandas Dataframe object in order to load it into MyDB. Use the *mydb_insert* function and the following parameters:  

*table* - name of the pre-existing MyDB table in which to insert the data  
*data* - location and name of the data to insert into the table

Let's first create some sample data and save it locally as a CSV file named `sampledata.csv`:

In [ ]:
sampledata_str = 'one,22.3,12.4,two,22.3,12.4,three,22.3,12.4,four,22.3,12.4,five,22.3,12.4'
with open ('sampledata.csv','w') as fd:
    fd.write (sampledata_str)

Now let's insert the `sampledata.csv` data into the `createdtable` table we created a few cells above:

In [ ]:
!datalab mydb_insert table="createdtable" data="./sampledata.csv"

**FROM A PYTHON SCRIPT (i.e. within this notebook, not on command line):** Let's make sure the data was inserted into the table by converting the table into a Pandas Dataframe and printing it on-screen:

In [ ]:
df1=convert(qc.query(sql="SELECT * FROM mydb://createdtable"))
df1

<a class="anchor" id="importcmdmydb"></a>
### 2.4 Importing data into a MyDB table
We can import data saved on a local computer or import data from VOSpace into a MyDB data table. The data must be in the form of either a CSV file or Pandas Dataframe object in order to load it into MyDB. Use the *mydb_import* function with the following parameters: 

*table* - name of the new MyDB table to create with the imported data  
*data* - location and name of the data to import  

Let's first query some example data from a Data Lab database and save it locally as a CSV file named `gaia_result.csv`:

In [8]:
query = "select * from gaia_dr1.gaia_source limit 10"
qc.query (adql=query, fmt='csv', out='./gaia_result.csv')

'OK'

Now we can import the queried data into a new MyDB table:

In [9]:
!datalab mydb_import table="gaia_result_table" data="./gaia_result.csv"

OK


Let's make sure the table was created in MyDB by calling the *mydb_list* function:

In [10]:
!datalab mydb_list

c4d,created:2020-08-27 14:55:07 MST
cd,created:2020-08-27 15:08:40 MST
createdtable,created:2020-09-04 17:00:56 MST
gaia_result_table,created:2020-09-04 17:06:05 MST
quickresults,created:2020-08-25 14:44:34 MST
quickresults2,created:2020-08-25 14:46:01 MST
usno_objects,created:2020-09-02 12:17:52 MST



**FROM A PYTHON SCRIPT (i.e. within this notebook, not on command line):** Let's also make sure the data was imported into the table by converting the table into a Pandas Dataframe and printing it on-screen:

In [11]:
df2=convert(qc.query(sql="SELECT * FROM mydb://gaia_result_table"))
df2

,pmdec_error,ra_dec_corr,solution_id,source_id,random_index,htm9,ring256,nest256,ref_epoch,ra,...,scan_direction_mean_k4,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b,ecl_lon,ecl_lat
0,NaN,-0.70700,380766208,5282605037517288,3802459346013519872,2882253,NaN,NaN,2015,219.386098,...,-23.588076,59,105.231307,2.212695,20.469408,NOT_AVAILABLE,325.929432,22.186421,228.387787,-19.485247
1,NaN,-0.60135,380766208,19155693000980960,3607672949823242240,2882253,NaN,NaN,2015,219.382135,...,-26.038597,51,2787.417442,5.019928,16.911765,NOT_AVAILABLE,325.926819,22.188809,228.384182,-19.485238
2,NaN,0.13645,380766208,5307344049142200,4709030434283978752,2882253,NaN,NaN,2015,219.393581,...,-24.180206,85,144.531456,1.828985,20.124864,NOT_AVAILABLE,325.937941,22.188877,228.391928,-19.477976
3,NaN,-0.99045,380766208,5247420665428392,4281789481637380096,2882242,NaN,NaN,2015,219.399990,...,-30.753368,98,6724.071722,5.852034,15.955689,NOT_AVAILABLE,325.934411,22.169914,228.403543,-19.493801
4,NaN,-0.81295,380766208,7754307176757673,2131676171190927360,2882253,NaN,NaN,2015,219.401910,...,-23.287327,69,306.952384,2.990556,19.307093,NOT_AVAILABLE,325.936628,22.170610,228.404580,-19.491865
5,NaN,-0.99565,380766208,5261714316589480,2263435610234552320,2882253,NaN,NaN,2015,219.408167,...,-31.125006,93,12291.791472,7.985610,15.300732,NOT_AVAILABLE,325.942983,22.171185,228.408608,-19.487332
6,NaN,-0.12870,380766208,7765302293035433,1164227542157099008,2882253,NaN,NaN,2015,219.409928,...,-23.373909,84,133.231945,1.712889,20.213249,NOT_AVAILABLE,325.943448,22.168769,228.410729,-19.488756
7,NaN,-0.89365,380766208,5256766514264488,3356372102641876992,2882253,NaN,NaN,2015,219.401041,...,-23.300661,68,1337.260831,12.173469,17.709230,NOT_AVAILABLE,325.936580,22.172155,228.403398,-19.490794
8,NaN,-0.03625,380766208,5277107479378688,1765130867442712576,2882253,NaN,NaN,2015,219.410698,...,-23.792971,86,207.345161,2.346311,19.733035,NOT_AVAILABLE,325.947960,22.176103,228.408443,-19.480594
9,NaN,-0.14785,380766208,5251268956125608,4898152713503637504,2882253,NaN,NaN,2015,219.416133,...,-23.608847,94,253.515705,2.048341,19.514758,NOT_AVAILABLE,325.947732,22.165413,228.416226,-19.488371


Similarly, we can use the *mydb_import* function to import data from VOSpace into a MyDB table:

In [39]:
!datalab mydb_import table="magsvos" data="vos://smags.csv"

OK


Let's make sure the table was created in MyDB by calling the *mydb_list* function:

In [40]:
!datalab mydb_list

c4d,created:2020-08-27 14:55:07 MST
cd,created:2020-08-27 15:08:40 MST
createdtable,created:2020-09-04 17:00:56 MST
gaia_result_table,created:2020-09-04 17:06:05 MST
magsvos,created:2020-09-04 18:01:23 MST
quickresults,created:2020-08-25 14:44:34 MST
quickresults2,created:2020-08-25 14:46:01 MST
usno_objects,created:2020-09-02 12:17:52 MST



We can view the schema definition of the table by calling the *mydb_list* function on the table:

In [41]:
!datalab mydb_list table="magsvos"

id,smallint,
g,double precision,
r,double precision,
i,double precision,



**FROM A PYTHON SCRIPT (i.e. within this notebook, not on command line):** Let's also make sure the data was imported into the table by converting the table into a Pandas Dataframe and printing it on-screen:

In [42]:
df3=convert(qc.query(sql="SELECT * FROM mydb://magsvos"))
df3

,id,g,r,i
0,1,22.3,12.4,21.5
1,2,22.3,12.4,21.5
2,3,22.3,12.4,21.5
3,4,22.3,12.4,21.5
4,5,22.3,12.4,21.5
5,6,22.3,12.4,21.5
6,7,22.3,12.4,21.5


<a class="anchor" id="truncmdmydb"></a>
### 2.5 Truncating a MyDB table   ????? 
**Does not truncate ... does not allow user to choose what or how much to truncate**

    class MyDB_Truncate(Task):
    '''
        Truncate a user MyDB table.
    '''
    def __init__(self, datalab):
        Task.__init__(self, datalab, 'mydb_truncate',
                      'Truncate a user MyDB table')
        self.addOption("table",
            Option("table", "", "Table name to create", required=True))
        self.addStdOptions()

    def run(self):
        token = getUserToken(self)
        try:
            res = queryClient.mydb_truncate (token, table=self.table.value)
        except Exception as e:
            print ("Error truncating table '%s': %s" % \
                   (self.table.value,str(e)))
        else:
            print (res)
            
We can truncate a MyDB table, i.e. drop all rows but keep the table definition (schema), with the *mydb_truncate* function and the following parameter:

*table* - name of the MyDB table to truncate

In [ ]:
!datalab mydb_truncate table="gaia_result_table" 

Let's make sure the table was truncated by calling the *mydb_list* function on the table: 

In [ ]:
!datalab mydb_list table="gaia_result_table"

**FROM A PYTHON SCRIPT (i.e. within this notebook, not on command line):** We can also make sure the table was truncated by converting the table into a Pandas Dataframe and printing it on-screen:

In [ ]:
df4=convert(qc.query(sql="SELECT * FROM mydb://gaia_result_table"))
df4

<a class="anchor" id="copycmdmydb"></a>
### 2.6 Copying a MyDB table
We can copy a MyDB table that currently exists in a user's MyDB database with the *mydb_copy* function and the following parameters:

*source* - name of table to copy  
*target* - name of new table with copied data from source table

In [43]:
!datalab mydb_copy source="magsvos" target="magsvos_copy"

OK


Let's make sure the newly copied table exists in MyDB database by calling the *mydb_list* function:

In [44]:
!datalab mydb_list

c4d,created:2020-08-27 14:55:07 MST
cd,created:2020-08-27 15:08:40 MST
createdtable,created:2020-09-04 17:00:56 MST
gaia_result_table,created:2020-09-04 17:06:05 MST
magsvos,created:2020-09-04 18:01:23 MST
magsvos_copy,
quickresults,created:2020-08-25 14:44:34 MST
quickresults2,created:2020-08-25 14:46:01 MST
usno_objects,created:2020-09-02 12:17:52 MST



<a class="anchor" id="renamecmdmydb"></a>
### 2.7 Renaming a MyDB table
We can choose to rename a MyDB table with the *mydb_rename* function and the following parameters:

*old* - name of table to rename  
*new* - new name of table

In [45]:
!datalab mydb_rename old="magsvos" new="newermagsvos"

OK


Let's make sure the name was changed by calling the *mydb_list* function:

In [46]:
!datalab mydb_list

c4d,created:2020-08-27 14:55:07 MST
cd,created:2020-08-27 15:08:40 MST
createdtable,created:2020-09-04 17:00:56 MST
gaia_result_table,created:2020-09-04 17:06:05 MST
magsvos_copy,
newermagsvos,created:2020-09-04 18:01:23 MST
quickresults,created:2020-08-25 14:44:34 MST
quickresults2,created:2020-08-25 14:46:01 MST
usno_objects,created:2020-09-02 12:17:52 MST



<a class="anchor" id="dropcmdmydb"></a>
### 2.8 Dropping a MyDB table
We can remove a MyDB table from a user's MyDB database by calling the *mydb_drop* function and the following parameter:

*table* - name of the table we wish to remove from MyDB database

In [48]:
!datalab mydb_drop table="newermagsvos"

OK


Let's make sure the MyDB table was dropped by calling the *mydb_list* function:

In [49]:
!datalab mydb_list

c4d,created:2020-08-27 14:55:07 MST
cd,created:2020-08-27 15:08:40 MST
createdtable,created:2020-09-04 17:00:56 MST
gaia_result_table,created:2020-09-04 17:06:05 MST
magsvos_copy,
quickresults,created:2020-08-25 14:44:34 MST
quickresults2,created:2020-08-25 14:46:01 MST
usno_objects,created:2020-09-02 12:17:52 MST



# Clean up MyDB
For clean-up purposes, let's remove the files we saved locally as well as the tables we created in MyDB and files/directories we created in VOSpace.

in MyDB
schema.txt
sampledata.csv
gaia_result.csv
createdtable
gaia_result_table
magsvos_copy

in VOSpace
smags.csv
mysmags.csv
vos://tmp